In [1]:
import os
import numpy as np
import tensorflow as tf

In [2]:
lexical_model = '/home/disooqi/qmdis-post-processor-full/arabic_dialect_identification/lexical/model'
acoustic_model = '/home/disooqi/qmdis-post-processor-full/arabic_dialect_identification/acoustic/model'

suwan_model = os.path.join(lexical_model, 'model60400.ckpt')
model_path = os.path.join(acoustic_model, 'model910000.ckpt')

In [3]:
class siamese:

    # Create model
    def __init__(self,input_dim):
        self.x1 = tf.placeholder(tf.float32, [None, input_dim])
        self.x2 = tf.placeholder(tf.float32, [None, input_dim])

        with tf.variable_scope("siamese") as scope:
            self.a1,self.b1,self.o1 = self.network(self.x1)
            scope.reuse_variables()
            self.a1,self.b2,self.o2 = self.network(self.x2)
            
        # Create loss
        self.y_ = tf.placeholder(tf.float32, [None])
        self.loss = self.loss_with_cds()

    def network(self, x):
        weights = []
        kernel_size =150
        stride = 18
        depth=40
        conv1 = self.conv_layer(x, kernel_size,stride,depth,'conv1')
        conv1r = tf.nn.relu(conv1)
        n_prev_weight = int(x.get_shape()[1])
        conv1_d = tf.reshape(conv1r,[-1, int(round(n_prev_weight/stride+0.5)*depth)])
        
        fc1 = self.fc_layer(conv1_d, 1500, "fc1")
        ac1 = tf.nn.relu(fc1)
        fc2 = self.fc_layer(ac1, 600, "fc2")   
        ac2 = tf.nn.relu(fc2)
        fc3 = self.fc_layer(ac2, 200, "fc3")
        return fc1,fc2,fc3

    def fc_layer(self, bottom, n_weight, name):
        # print( bottom.get_shape())
        n_prev_weight = bottom.get_shape()[1]
        W = tf.get_variable(name+'W', dtype=tf.float32, shape=[n_prev_weight, n_weight], initializer=tf.contrib.layers.xavier_initializer())
        b = tf.get_variable(name+'b', dtype=tf.float32, initializer=tf.random_uniform([n_weight],-0.001,0.001, dtype=tf.float32))
        fc = tf.nn.bias_add(tf.matmul(bottom, W), b)
        return fc

    def conv_layer(self, bottom, kernel_size, stride, depth, name):        
        n_prev_weight = int(bottom.get_shape()[1])
        num_channels = 1 # for 1 dimension
        inputlayer = tf.reshape(bottom, [-1,n_prev_weight,1])
        initer = tf.truncated_normal_initializer(stddev=0.1)
        W = tf.get_variable(name+'W', dtype=tf.float32, shape=[kernel_size, num_channels, depth], initializer=tf.contrib.layers.xavier_initializer())
        b = tf.get_variable(name+'b', dtype=tf.float32, initializer=tf.constant(0.001, shape=[depth*num_channels], dtype=tf.float32))
        
        conv = tf.nn.bias_add( tf.nn.conv1d(inputlayer, W, stride, padding='SAME'), b)
        return conv

    def loss_with_cds(self):
        labels_t = self.y_
        cds = tf.reduce_sum(tf.multiply(self.o1,self.o2),1)
        eucd2 = tf.reduce_mean(tf.pow(tf.subtract(labels_t,cds),2))
        eucd = tf.sqrt(eucd2, name="eucd")
        return eucd

In [4]:
graph_1 = tf.Graph()

with graph_1.as_default():
    input_dim = 41657
    siamese = siamese(input_dim)
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(0.01, global_step, 5000, 0.99, staircase=True)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(siamese.loss, global_step=global_step)
    saver = tf.train.Saver()

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    saver.restore(sess, suwan_model)

INFO:tensorflow:Restoring parameters from /home/disooqi/qmdis-post-processor-full/arabic_dialect_identification/lexical/model/model60400.ckpt


In [5]:
x = tf.placeholder(tf.float32, [None,None,40])
y = tf.placeholder(tf.int32, [None])
s = tf.placeholder(tf.int32, [None,2])

softmax_num = 5

In [6]:
class nn:
    # Create model
    def __init__(self, x1, y_, y_string, shapes_batch, softmax_num):
        self.ea, self.eb, self.o1,self.res1,self.conv,self.ac1,self.ac2 = self.net(x1, shapes_batch, softmax_num)
            
        # Create loss
        self.loss    = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_, logits=self.o1))
        self.label=y_
        self.shape = shapes_batch
        self.true_length = x1
        self.label_string=y_string

    def net(self,x, shapes_batch,softmax_num):   
        
        shape_list = shapes_batch[:,0]

        featdim = 40 #channel
        weights = []
        kernel_size =5
        stride = 1
        depth = 500
                
        shape_list = shape_list/stride
        conv1 = self.conv_layer(x,kernel_size,featdim,stride,depth,'conv1',shape_list)
        conv1r= tf.nn.relu(conv1)
       

        featdim = depth #channel
        weights = []
        kernel_size =7
        stride = 2
        depth = 500
                
        shape_list = shape_list/stride
        conv2 = self.conv_layer(conv1r,kernel_size,featdim,stride,depth,'conv2',shape_list)
        conv2r= tf.nn.relu(conv2)
       
        featdim = depth #channel
        weights = []
        kernel_size =1
        stride = 1
        depth = 500
                
        shape_list = shape_list/stride
        conv3 = self.conv_layer(conv2r,kernel_size,featdim,stride,depth,'conv3',shape_list)
        conv3r= tf.nn.relu(conv3)
       
        featdim = depth #channel
        weights = []
        kernel_size =1
        stride = 1
        depth = 3000
                
        shape_list = shape_list/stride
        conv4 = self.conv_layer(conv3r,kernel_size,featdim,stride,depth,'conv4',shape_list)
        conv4r= tf.nn.relu(conv4)
        
        print ('Hi I am conv1', conv1)
        
        shape_list = tf.cast(shape_list, tf.float32)
        shape_list = tf.reshape(shape_list,[-1,1,1])
        mean = tf.reduce_sum(conv4r,1,keep_dims=True)/shape_list
        res1=tf.squeeze(mean,axis=1)
        

        fc1 = self.fc_layer(res1,1500,"fc1")
        ac1 = tf.nn.relu(fc1)
        fc2 = self.fc_layer(ac1,600,"fc2")
        ac2 = tf.nn.relu(fc2)
        
        fc3 = self.fc_layer(ac2,softmax_num,"fc3")
        return fc1, fc2, fc3,res1,conv1r,ac1,ac2
        
    def xavier_init(self,n_inputs, n_outputs, uniform=True):
        if uniform:
            init_range = np.sqrt(6.0 / (n_inputs + n_outputs))
            return tf.random_uniform_initializer(-init_range, init_range)
        else:
            stddev = np.sqrt(3.0 / (n_inputs + n_outputs))
            return tf.truncated_normal_initializer(stddev=stddev)

    def fc_layer(self, bottom, n_weight, name):
        print(bottom.get_shape())
        assert len(bottom.get_shape()) == 2
        n_prev_weight = bottom.get_shape()[1]

        initer = self.xavier_init(int(n_prev_weight),n_weight)
        W = tf.get_variable(name+'W', dtype=tf.float32, shape=[n_prev_weight, n_weight], initializer=initer)
        b = tf.get_variable(name+'b', dtype=tf.float32, initializer=tf.random_uniform([n_weight],-0.001,0.001, dtype=tf.float32))
        fc = tf.nn.bias_add(tf.matmul(bottom, W), b)
        return fc
    
    
    def conv_layer(self, bottom, kernel_size,num_channels, stride, depth, name, shape_list):   # n_prev_weight = int(bottom.get_shape()[1])
        n_prev_weight = tf.shape(bottom)[1]

        inputlayer=bottom
        initer = tf.truncated_normal_initializer(stddev=0.1)

        W = tf.get_variable(name+'W', dtype=tf.float32, shape=[kernel_size, num_channels, depth], initializer=tf.contrib.layers.xavier_initializer())
        b = tf.get_variable(name+'b', dtype=tf.float32, initializer=tf.constant(0.001, shape=[depth], dtype=tf.float32))

        conv =  ( tf.nn.bias_add( tf.nn.conv1d(inputlayer, W, stride, padding='SAME'), b))
        mask = tf.sequence_mask(shape_list,tf.shape(conv)[1]) # make mask with batch x frame size
        mask = tf.where(mask, tf.ones_like(mask,dtype=tf.float32), tf.zeros_like(mask,dtype=tf.float32))
        mask=tf.tile(mask, tf.stack([tf.shape(conv)[2],1])) #replicate make with depth size
        mask=tf.reshape(mask,[tf.shape(conv)[2], tf.shape(conv)[0], -1])
        mask = tf.transpose(mask,[1, 2, 0])
        print('Hi I am mask', mask)
        conv=tf.multiply(conv,mask)
        return conv
    


In [7]:
emnet_validation = nn(x,y,y,s,softmax_num)
sess2 = tf.Session()
saver2 = tf.train.Saver()
sess2.run(tf.global_variables_initializer())
saver2.restore(sess2, model_path)


Hi I am mask Tensor("transpose:0", shape=(?, ?, ?), dtype=float32)
Hi I am mask Tensor("transpose_1:0", shape=(?, ?, ?), dtype=float32)
Hi I am mask Tensor("transpose_2:0", shape=(?, ?, ?), dtype=float32)
Hi I am mask Tensor("transpose_3:0", shape=(?, ?, ?), dtype=float32)
Hi I am conv1 Tensor("Mul:0", shape=(?, ?, 500), dtype=float32)
Instructions for updating:
keep_dims is deprecated, use keepdims instead
(?, 3000)
(?, 1500)
(?, 600)
INFO:tensorflow:Restoring parameters from /home/disooqi/qmdis-post-processor-full/arabic_dialect_identification/acoustic/model/model910000.ckpt
